# Libraries and Setup

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
from copy import deepcopy
from astropy.cosmology import LambdaCDM
import sys; sys.path.insert(0,'/srv/one/zhutchen/paper3/codes/')
from prob_g3groupfinder import prob_g3groupfinder_luminosity

basepath = "/srv/one/zhutchen/paper3/figures/"
figure4filename = "figure4_rproj_vproj_cal.pdf"
figure5filename = "figure5_gdrproj_gdvproj_cal.pdf"
photoz_fraction = 0.85
prob_thresh = 0.6
nominal_specz_error = 35 # km/s
hubble_const = 70.
omega_m = 0.3
omega_de = 0.7

ecovolume = 191958.08 / (hubble_const/100.)**3.
gfargseco = dict({'volume':ecovolume,'rproj_fit_multiplier':3,'vproj_fit_multiplier':4,'vproj_fit_offset':200,'showplots':True,'saveplotspdf':True,
       'gd_rproj_fit_multiplier':2, 'gd_vproj_fit_multiplier':4, 'gd_vproj_fit_offset':100,\
       'gd_fit_bins':np.arange(-24,-19,0.25), 'gd_rproj_fit_guess':[1e-5, 0.4],\
       'pfof_Pth' : prob_thresh, \
       'gd_vproj_fit_guess':[3e-5,4e-1], 'H0':hubble_const, 'Om0':omega_m, 'Ode0':omega_de,  'iterative_giant_only_groups':True})

# Prepare input data (ECO w/ photo-z mix)

In [11]:
eco = pd.read_csv("/srv/one/zhutchen/g3groupfinder/resolve_and_eco/ECOdata_G3catalog_luminosity.csv")
eco = eco[eco.absrmag<-17.33] # just to test
eco.loc[:,'czerr'] = eco.cz*0 + 20
ecophotz = pd.read_csv("/srv/one/hperk4/eco_resb_decals_photoz.csv")
ecophotz = ecophotz[ecophotz.name.str.startswith('ECO')].set_index('name')
eco = pd.concat([eco,ecophotz],axis=1)
eco.loc[:,'photo_z_corr'] = eco.photo_z_corr.fillna(value=eco.cz)
eco.loc[:,'e_tab_corr'] = eco.e_tab_corr.fillna(value=eco.czerr)

In [12]:
degradedcz = deepcopy(eco.cz.to_numpy())
zphot = deepcopy(eco.photo_z_corr.to_numpy())
zphoterr = deepcopy(eco.e_tab_corr.to_numpy())
degradedczerr = np.zeros_like(degradedcz)+35

idx = np.random.choice(np.indices(degradedcz.shape)[0], size=int(photoz_fraction*len(degradedcz)), replace=False) # originally 0.85
degradedcz[idx] = zphot[idx]
degradedczerr[idx] = zphoterr[idx]


sel = np.isnan(degradedcz)
degradedcz[sel] = eco.cz.to_numpy()[sel]
degradedczerr[sel] = nominal_specz_error

eco.loc[:,'degradedcz'] = degradedcz
eco.loc[:,'degradedczerr'] = degradedczerr

# Do group-finding

In [13]:
t1 = time.time()
cosmo=LambdaCDM(hubble_const, omega_m, omega_de)
pg3out=prob_g3groupfinder_luminosity(eco.radeg, eco.dedeg, eco.degradedcz, eco.degradedczerr, eco.absrmag,-19.5,fof_bperp=0.07,fof_blos=1.1,**gfargseco)
pg3grp=pg3out[0]
eco.loc[:,'pg3grp'] = pg3grp
print('elapsed time was ', time.time()-t1)

PFoF complete!
Giant-only iterative combination 0 in progress...


/afs/cas.unc.edu/users/z/h/zhutchen/.conda/envs/astro2/lib/python3.10/site-packages/scipy/integrate/_quadpack_py.py:1225: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
/afs/cas.unc.edu/users/z/h/zhutchen/.conda/envs/astro2/lib/python3.10/site-packages/scipy/integrate/_quadpack_py.py:1225: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  quad_r = quad(f, low, high, args=args, full_output=self.full

Giant-only iterative combination 1 in progress...
Giant-only iterative combination 2 in progress...
Giant-only iterative combination complete.
Finished associating dwarfs to giant-only groups.
Beginning iterative combination...
iteration 0 in progress...


/afs/cas.unc.edu/users/z/h/zhutchen/.conda/envs/astro2/lib/python3.10/site-packages/scipy/integrate/_quadpack_py.py:1225: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
/afs/cas.unc.edu/users/z/h/zhutchen/.conda/envs/astro2/lib/python3.10/site-packages/scipy/integrate/_quadpack_py.py:1225: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  quad_r = quad(f, low, high, args=args, full_output=self.full

iteration 1 in progress...
Iterative combination complete.
elapsed time was  754.3903720378876


In [18]:
os.system(f"mv ../figures/rproj_vproj_cal.pdf ../figures/{figure4filename}")
os.system(f"mv ../figures/itercombboundaries.pdf ../figures/{figure5filename}")

0

# Multiplicity Function

In [ ]:
# bins = np.arange(0.5,300.5,1)
# plt.figure()
# plt.hist(fof.multiplicity_function(eco.g3grp_l.to_numpy(), return_by_galaxy=False), bins=bins, color='gray', histtype='stepfilled', label='G3 Groups', alpha=0.7)
# plt.hist(fof.multiplicity_function(eco.pg3grp, return_by_galaxy=False), bins=bins, color='green', histtype='step', label='PG3 Groups', linewidth=3)
# plt.yscale('log')
# plt.xlabel("Group N")
# plt.xlim(0,50)
# plt.legend(loc='best')
# plt.show()